In [ ]:
# Подключение модулей
from keras.models import Sequential
from keras.layers.core import Activation, Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from keras.applications.xception import Xception, preprocess_input
import cv2, numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Пути к наборам данных
train_dir = '/content/drive/MyDrive/Keras/horse-or-human/Test'
test_dir = '/content/drive/MyDrive/Keras/horse-or-human/Validation' 
# Размер изображений
img_width, img_height = 300, 300
# Размер мини-выборки
batch_size = 64
# Кол-во изображений для обучения
nb_train_samples = 1027
# Кол-во изображений для теста
nb_test_samples = 256

In [ ]:
# Создание генератора изображений
datagen = ImageDataGenerator(rescale=1. / 255)

# Генератор для обучения
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

# Генератор для теста
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [ ]:
# Создание экземпляра модели сети Xception
xception = Xception(
    weights='imagenet', 
    include_top=False, 
    input_shape=(img_width, img_height, 3))
# weights - веса предварительно обученной сети
# include_top = false означает, что мы загружаем только сверточную часть сети, без квалификационной
# input_shape - размер тензора

# Сверточную часть сети обучать не надо
xception.trainable = False

xception.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 300, 300, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_9[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [ ]:
# Создание модели составной сети
model = Sequential()
# Добавляем сверточные слои
model.add(xception)
# Преобразуем двумерный массив Xception в одномерный
model.add(Flatten())
# Полносвязный слой
model.add(Dense(256, activation='relu'))
# Слой регуляризации (для предотвращения переобучения)
model.add(Dropout(0.5))
# Кол-во классов
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 10, 10, 2048)      20861480  
_________________________________________________________________
flatten_11 (Flatten)         (None, 204800)            0         
_________________________________________________________________
dense_22 (Dense)             (None, 256)               52429056  
_________________________________________________________________
dropout_11 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 257       
Total params: 73,290,793
Trainable params: 52,429,313
Non-trainable params: 20,861,480
_________________________________________________________________


In [ ]:
# Компилируем составную сеть
model.compile(loss='binary_crossentropy',
              optimizer='SGD', 
              metrics=['accuracy'])

In [ ]:
# Обучение
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=7,
    validation_data=test_generator,
    validation_steps=nb_test_samples // batch_size)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/7
16/16 [==============================] - 31s 938ms/step - loss: 0.1803 - accuracy: 0.8816 - val_loss: 0.0114 - val_accuracy: 1.0000
Epoch 2/7
16/16 [==============================] - 12s 800ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.0067 - val_accuracy: 1.0000
Epoch 3/7
16/16 [==============================] - 12s 754ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.0059 - val_accuracy: 1.0000
Epoch 4/7
16/16 [==============================] - 12s 749ms/step - loss: 8.1999e-04 - accuracy: 1.0000 - val_loss: 0.0038 - val_accuracy: 1.0000
Epoch 5/7
16/16 [==============================] - 12s 753ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.0037 - val_accuracy: 1.0000
Epoch 6/7
16/16 [==============================] - 12s 746ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.0028 - val_accuracy: 1.0000
Epoch 7/7
16/16 [==============================] - 12s 735ms/step - loss: 6.5282e-04 - accuracy: 1.0000 - val_loss: 0.0042 - val_accuracy: 1.000

In [ ]:
# Проверяем модель на тестовой выборке
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print('Точность работы на тестовых данных: %.2f%%' % (scores[1]*100))

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1948: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


Точность работы на тестовых данных: 100.00%
